# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas ./lab/data/Data.csv

In [19]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [20]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [21]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [22]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)


In [23]:
prompt = ChatPromptTemplate.from_template( "Describe the features and benefits of the product: {product}")

In [24]:

chain = LLMChain(llm=llm, prompt=prompt)

In [27]:
product = "Queen Size Sheet Set"
response = chain.run(product)

In [28]:
print("Description for product:", response)

Description for product: The Queen Size Sheet Set is a luxurious bedding set designed to fit a standard queen size bed. 

Features:
1. The set includes a flat sheet, a fitted sheet, and two pillowcases, all made from high-quality, soft and breathable fabric.
2. The sheets are designed to have a deep pocket fitted sheet to ensure a snug fit on your mattress.
3. The set is available in a variety of colors and patterns to complement any bedroom decor.
4. The sheets are easy to care for and are machine washable for convenience.
5. The fabric is hypoallergenic, making it suitable for sensitive skin.

Benefits:
1. The Queen Size Sheet Set provides a comfortable and cozy sleeping environment, promoting a restful night's sleep.
2. The deep pocket fitted sheet ensures that the sheets stay in place throughout the night, preventing bunching or slipping off the mattress.
3. The variety of colors and patterns available allow you to customize your bedding to suit your personal style.
4. The easy-car

## SimpleSequentialChain

In [29]:
from langchain.chains import SimpleSequentialChain

In [30]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template("Describe the features and benefits of the product: {product}")

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [31]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template("Using the features: {input}, write a slogan for this product.")
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [32]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [33]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
A queen size sheet set typically includes one flat sheet, one fitted sheet, and two pillowcases, all designed to fit a standard queen size mattress. 

Features of a queen size sheet set may include:
- Made from high-quality, soft and durable materials such as cotton, microfiber, or bamboo
- Available in a variety of colors and patterns to suit any decor
- Fitted sheet with deep pockets to ensure a secure fit on the mattress
- Pillowcases with envelope closures to keep pillows in place
- Machine washable for easy care and maintenance

Benefits of a queen size sheet set may include:
- Ensures a comfortable and cozy sleep environment
- Provides a smooth and soft surface for a restful night's sleep
- Easy to care for and keep clean
- Helps to protect and extend the life of your mattress
- Adds a decorative touch to your bedroom

Overall, a queen size sheet set is a practical and essential bedding item that enhances the comfort and aesthetics 

'"Sleep in style and comfort with our luxurious queen size sheet sets!"'

**Repeat the above twice for different products**

In [34]:
# Run the chain for a product
product = "	Waterproof Phone Pouch"
result = overall_simple_chain.run(product)
print("Write a slogan for this product:", result)

# Repeat for another product
product = "Pillows Insert"
result = overall_simple_chain.run(product)
print("Write a slogan for this product:", result)



> Entering new SimpleSequentialChain chain...
The waterproof phone pouch is a protective case that keeps your phone safe from water damage. Some key features and benefits of this product include:

1. Waterproof protection: The pouch is designed to keep your phone safe from water, making it ideal for use at the beach, pool, or on rainy days.

2. Touchscreen compatibility: The pouch is made of a transparent material that allows you to still use your phone's touchscreen while it is inside the case.

3. Universal fit: The pouch is designed to fit most smartphones, including iPhones and Android devices, up to a certain size.

4. Adjustable strap: The pouch comes with an adjustable strap that allows you to wear it around your neck or shoulder for easy access.

5. Multi-functional: In addition to protecting your phone from water, the pouch can also protect it from dust, dirt, and sand.

Overall, the waterproof phone pouch is a convenient and practical accessory for anyone who wants to keep 

## SequentialChain

In [35]:
from langchain.chains import SequentialChain

In [50]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate the review into the japanese language: {review}")

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key= "translated_review"
                    )


In [51]:
second_prompt = ChatPromptTemplate.from_template("Summarize this review into one sentence: {translated_review}")

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key= "summarized_review"
                    )


In [52]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template("Translate the summary into the french language: {summarized_review}"
    
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="french_summary"
                      )


In [53]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template("Write a follow-up message to the customer based on this summary: {french_summary}")
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="follow_up_message"
                     )


In [54]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["translated_review", "summarized_review", "french_summary", "follow_up_message"],
    verbose=True
)

In [55]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'translated_review': '味があまり良くないと感じます。泡が立たないのは変です。同じ商品を買うことができますが、味がはるかに良いです...古いロットか偽物！？',
 'summarized_review': 'The taste is not very good and the lack of bubbles is strange, suggesting that the product may be old or fake.',
 'french_summary': "Le goût n'est pas très bon et l'absence de bulles est étrange, ce qui suggère que le produit pourrait être ancien ou faux.",
 'follow_up_message': "Bonjour,\n\nNous sommes désolés d'apprendre que vous n'êtes pas satisfait(e) de votre produit. Nous prenons votre feedback très au sérieux et nous voulons résoudre ce problème pour vous. Pourriez-vous nous fournir plus de détails sur votre expérience afin que nous puissions mieux comprendre la situation?\n\nNous tenons à vous assurer que nos produits sont frais et authentiques, mais nous allons enquêter sur cette question 

**Repeat the above twice for different products or reviews**

## Router Chain

In [56]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [57]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [58]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [59]:
llm = ChatOpenAI(temperature=0)

In [60]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [61]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [62]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [63]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [64]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [70]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [71]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [72]:
chain.run("Why does every cell in our body contain DNA?")
#print("Biology response:", result)



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [74]:
chain.run("What is the time complexity of quicksort?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is the time complexity of quicksort?'}
> Finished chain.


'The time complexity of quicksort is O(n log n) on average, where n is the number of elements in the array being sorted. This is because quicksort divides the array into smaller subarrays and recursively sorts them, with each division taking O(n) time and there being O(log n) divisions. However, in the worst-case scenario, quicksort can have a time complexity of O(n^2) if the pivot element chosen is consistently the smallest or largest element in the array.'

In [75]:
chain.run("What happened during 1939 in Germany?")




> Entering new MultiPromptChain chain...
History: {'input': 'What happened during 1939 in Germany?'}
> Finished chain.


'In 1939, Germany, under the leadership of Adolf Hitler and the Nazi Party, invaded Poland on September 1st, marking the beginning of World War II. This invasion led to the declaration of war by Britain and France against Germany. The German military quickly advanced through Poland, using blitzkrieg tactics, and within a few weeks, Poland was divided between Germany and the Soviet Union as per the Molotov-Ribbentrop Pact.\n\nAdditionally, in 1939, Germany also signed the Pact of Steel with Italy, forming the Axis Powers. This alliance would later expand to include Japan, leading to a global conflict.\n\nOverall, 1939 was a significant year for Germany as it marked the beginning of World War II and the further expansion of Nazi influence in Europe.'